# customer_hm

## Prepare

In [1]:
import pandas as pd
customer = pd.read_csv("../../main/h&m_dataset/customer_hm.csv")
customer.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,NONE,49
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,ACTIVE,NONE,25
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,ACTIVE,NONE,24
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,0,ACTIVE,NONE,54
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,1,ACTIVE,Regularly,52


In [2]:
customer.info()

<class 'pandas.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column                  Non-Null Count    Dtype
---  ------                  --------------    -----
 0   customer_id             1048575 non-null  str  
 1   FN                      1048575 non-null  int64
 2   Active                  1048575 non-null  int64
 3   club_member_status      1048575 non-null  str  
 4   fashion_news_frequency  1048574 non-null  str  
 5   age                     1048575 non-null  int64
dtypes: int64(3), str(3)
memory usage: 48.0 MB


In [3]:
customer.shape

(1048575, 6)

In [4]:
na_c = customer.isna().sum().sort_values(ascending=False) 
na_c.head(3)

fashion_news_frequency    1
customer_id               0
FN                        0
dtype: int64

In [5]:
customer["age"].describe().T[["min", "max"]]

min    16.0
max    99.0
Name: age, dtype: float64

In [6]:
customer["FN"].value_counts(dropna=False)

FN
0    675765
1    372810
Name: count, dtype: int64

In [7]:
customer["Active"].value_counts(dropna=False)

Active
0    685629
1    362946
Name: count, dtype: int64

In [8]:
customer["club_member_status"].value_counts(dropna=False)

club_member_status
ACTIVE        982635
PRE-CREATE     65581
LEFT CLUB        359
Name: count, dtype: int64

In [9]:
customer["fashion_news_frequency"].value_counts(dropna=False)

fashion_news_frequency
NONE         674698
Regularly    373218
Monthly         658
NaN               1
Name: count, dtype: int64

## 고객 1명당 1행으로 구성된 고객 특성 테이블
- 직접적인 매출 정보가 없기 때문에 거래 데이터와의 결합이 필요

## Prepare 결과
- customer_id                   # 식별자 (조인 키)
- FN                            # 패션 뉴스 수신 여부
- Active                        # 활성 고객 여부
- club_member_status            # 클럽 가입 상태 (신규 / 활성 / 탈퇴)
- fashion_news_frequency        # 뉴스 알림 주기
- age                           # 연령대 분석



### 결측치 여부
- fashion_news_frequency에 결측치가 존재하긴 하나 1개 존재.
    - 별도의 카테고리로 구성

### 형변환
- FN, Active의 값은 0/1 로 구성되어 있다. 불리언(참/거짓)으로 분리

### 중복여부

### 이상치 여부
데이터 내에서 age의 범위는 16~99로 정상범주에 속한다 볼수 있다. <br>
튀는 값 자체는 존재하나 아직 전처리 초기 단계이고 값들이 상식선 안에 있기 때문에 유지한다.

# transactions_hm

## Prepare

In [10]:
import pandas as pd
transactions = pd.read_csv("../../main/h&m_dataset/transactions_hm.csv")
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2019-11-05,3e2b60b679e62fb49516105b975560082922011dd752ec...,698328010,0.016932,2
1,2019-05-22,89647ac2274f54c770aaa4b326e0eea09610c252381f37...,760597002,0.033881,2
2,2019-05-10,2ebe392150feb60ca89caa8eff6c08b7ef1138cd6fdc71...,488561032,0.016932,2
3,2019-08-26,7b3205de4ca17a339624eb5e3086698e9984eba6b47c56...,682771001,0.033881,2
4,2019-08-10,3b77905de8b32045f08cedb79200cdfa477e9562429a39...,742400033,0.003220,1


In [11]:
transactions.info()

<class 'pandas.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   t_dat             1048575 non-null  str    
 1   customer_id       1048575 non-null  str    
 2   article_id        1048575 non-null  int64  
 3   price             1048575 non-null  float64
 4   sales_channel_id  1048575 non-null  int64  
dtypes: float64(1), int64(2), str(2)
memory usage: 40.0 MB


In [12]:
transactions.shape

(1048575, 5)

In [13]:
na_t = transactions.isna().sum().sort_values(ascending=False) 
na_t.head(3)

t_dat          0
customer_id    0
article_id     0
dtype: int64

In [14]:
transactions["sales_channel_id"].value_counts(dropna=False)

sales_channel_id
2    729192
1    319383
Name: count, dtype: int64

In [15]:
transactions.duplicated().sum()

np.int64(8474)

## 특정 고객이, 날짜에, 상품을, 가격을, 온/오프라인 에서 구매한 기록의 테이블 
고객 및 상품 테이블과의 결합을 통해 고객특성·상품군·채널 분석이 가능하다. 
- 한 행이 한 번의 구매 기록
- customer_id                   # 구매한 고객 → customers_hm과 연결
- article_id                    # 구매한 상품 → articles_hm과 연결
- t_dat                         # 구매날짜
- price                         # 구매 가격 == 매출
- sales_channel_id              # 판매 채널 구분 (1=오프라인, 2=온라인)


### 결측치 여부
- 존재하지 않음

### 형변환
- t_dat는 문자열로 들어와있기 때문에 datetime변환이 필수
- 월별, 요일별 파생 시도 가능

### 이상치 여부
- 값 대부분이 소수점으로 통화 단위가 SEK 인것 때문으로 보임
    - 이 때문에 해석의 혼동이 발생할수 있음 (따로 지정된 단위가 있는지 확인하기)
    - 컬럼명을 revenue_sek로 변경

### 중복행 여부
완전 중복행이 8474 존재 어떻게 처리해야할지 미정

### price
매출 금액은 SEK 기준 거래 단가 합으로, 절대 금액보다는 고객 간 상대적 비교에 초점을 둔다.

# articles_hm

## Prepare

In [16]:
import pandas as pd
articles = pd.read_csv("../../main/h&m_dataset/articles_hm.csv")
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [17]:
articles.info()

<class 'pandas.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   article_id                    105542 non-null  int64
 1   product_code                  105542 non-null  int64
 2   prod_name                     105542 non-null  str  
 3   product_type_no               105542 non-null  int64
 4   product_type_name             105542 non-null  str  
 5   product_group_name            105542 non-null  str  
 6   graphical_appearance_no       105542 non-null  int64
 7   graphical_appearance_name     105542 non-null  str  
 8   colour_group_code             105542 non-null  int64
 9   colour_group_name             105542 non-null  str  
 10  perceived_colour_value_id     105542 non-null  int64
 11  perceived_colour_value_name   105542 non-null  str  
 12  perceived_colour_master_id    105542 non-null  int64
 13  perceived_colour_master_n

In [18]:
articles.shape

(105542, 25)

In [19]:
na_a = articles.isna().sum().sort_values(ascending=False) 
na_a.head(3)

detail_desc     416
product_code      0
prod_name         0
dtype: int64

In [20]:
articles.duplicated().sum()

np.int64(0)

In [21]:
articles["article_id"].duplicated().sum()

np.int64(0)

In [22]:
articles.describe().T[["min", "max"]]

,min,max
article_id,108775015.0,959461001.0
product_code,108775.0,959461.0
product_type_no,-1.0,762.0
graphical_appearance_no,-1.0,1010029.0
colour_group_code,-1.0,93.0
perceived_colour_value_id,-1.0,7.0
perceived_colour_master_id,-1.0,20.0
department_no,1201.0,9989.0
index_group_no,1.0,26.0
section_no,2.0,97.0


In [23]:
articles["product_group_name"].value_counts(dropna=False)

product_group_name
Garment Upper body       42741
Garment Lower body       19812
Garment Full body        13292
Accessories              11158
Underwear                 5490
Shoes                     5283
Swimwear                  3127
Socks & Tights            2442
Nightwear                 1899
Unknown                    121
Underwear/nightwear         54
Cosmetic                    49
Bags                        25
Items                       17
Furniture                   13
Garment and Shoe care        9
Stationery                   5
Interior textile             3
Fun                          2
Name: count, dtype: int64

In [24]:
articles["garment_group_name"].value_counts(dropna=False)

garment_group_name
Jersey Fancy                     21445
Accessories                      11519
Jersey Basic                      8126
Knitwear                          7490
Under-, Nightwear                 7441
Trousers                          6727
Blouses                           5838
Shoes                             5145
Dresses Ladies                    4874
Outdoor                           4501
Unknown                           3873
Trousers Denim                    3100
Swimwear                          2787
Socks and Tights                  2272
Shirts                            2116
Woven/Jersey/Knitted mix Baby     1965
Shorts                            1559
Dresses/Skirts girls              1541
Skirts                            1254
Special Offers                    1061
Dressed                            908
Name: count, dtype: int64

In [25]:
articles["section_name"].value_counts(dropna=False)

section_name
Womens Everyday Collection        7295
Divided Collection                7124
Baby Essentials & Complements     4932
Kids Girl                         4469
Young Girl                        3899
Womens Lingerie                   3598
Girls Underwear & Basics          3490
Womens Tailoring                  3376
Kids Boy                          3328
Womens Small accessories          3270
Womens Casual                     2725
Kids Outerwear                    2665
Womens Trend                      2622
Divided Projects                  2364
Young Boy                         2352
H&M+                              2337
Men Underwear                     2322
Mama                              2266
Kids & Baby Shoes                 2142
Boys Underwear & Basics           2034
Womens Shoes                      2026
Ladies H&M Sport                  1894
Womens Swimwear, beachwear        1839
Contemporary Smart                1778
Baby Girl                         1760
Divided Acce

In [26]:
articles["department_name"].value_counts(dropna=False)

department_name
Jersey                             4604
Knitwear                           3503
Trouser                            2655
Blouse                             2362
Dress                              2087
                                   ... 
Shirt Extended inactive from s1       1
Kids Boy License                      1
Jersey inactive from S.6              1
EQ Divided Blue                       1
Accessories Other                     1
Name: count, Length: 250, dtype: int64

In [27]:
articles["graphical_appearance_name"].value_counts().head(10)

graphical_appearance_name
Solid               49747
All over pattern    17165
Melange              5938
Stripe               4990
Denim                4842
Front print          3215
Placement print      3098
Check                2178
Colour blocking      1830
Lace                 1513
Name: count, dtype: int64

## Prepare 결과

### 결측치 여부
- detail_desc에 존재
    - 제품에 대한 설명일 가능성이 높기 때문에 배제

### 형변환
- t_dat는 문자열로 들어와있기 때문에 datetime변환이 필수
- 월별, 요일별 파생 시도 가능

### 이상치 여부
- 코드값이 -인 게 몇개 존재함

### 중복행 여부
존재하지 않음

- article_id                    # 상품을 식별하는 ID
- product_code                  # 상품 코드
- prod_name                     # 상품 이름

- 상품 분류
    - product_type_no	            # 상품 유형 코드
    - product_type_name	            # 상품 유형 명 - “T-shirt”, “Dress”
    - product_group_name            # 상품 대분류 - “Garment Upper body”

- 상품군
    - garment_group_no	            # 의류 그룹 코드
    - garment_group_name	        # 의류 그룹 이름 - “Jersey Basic”

- 타겟 그룹 코드
    - index_group_no                # 판매 인덱스 그룹 코드
    - index_group_name              # 성별·연령대 기반 그룹 - “Ladieswear”, “Menswear”

- 중분류
    - index_code                    # 중분류 코드
    - index_name                    # 중분류 명 (예: Lingeries/Tights)

- 판매 섹션
    - section_no                    # 섹션 코드
    - section_name                  # 실제 판매 섹션 - “Womens Everyday Basics”

- 부서
    - department_no                 # 내부 부서 코드
    - department_name               # 부서 이름 - 보조적 상품 분류

- 디자인
    - graphical_appearance_no       # 외형 정보 코드 
    - graphical_appearance_name     # 외형 스타일 - “Solid”, “All over pattern”

- 색상
    - colour_group_code             # 색상 그룹 코드
    - colour_group_name             # 상품 색상 명 - “Black”, “White”

- 명도
    - perceived_colour_value_id     # 색 밝기/톤 코드 - 명도 분류
    - perceived_colour_value_name   # 밝기 느낌 - “Dark”, “Light”

- 색 계열 코드
    - perceived_colour_master_id    # 색 계열 코드
    - perceived_colour_master_name  # 색 계열 이름 - “Black”, “Blue”

- detail_desc                   # 상품 상세 설명
- 버릴 데이터


- 제거할 컬럼
- *_no
- *_code
- prod_name
- department_*